<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Using FAST-OAD for the sizing and study of the aircraft

This notebooks aims to show you how FAST-OAD can be used to analyse the design of an aircraft and its performance on other mission than the design one. It will also aims to start a reflection on the performance difference between an aircraft used on a shuttle mission and an aircraft used on its design mission.

***
***
## Imports

In [1]:
import os.path as pth
import sys
import logging
import shutil
import warnings

warnings.filterwarnings(action='ignore')

import openmdao.api as om

import fastoad.api as oad

sys.path.append(pth.abspath("."))

## Working directories and reference files 
Here you can define your working directory and data storage folder as well as reference files names. 

In [3]:
logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

DATA_FOLDER_PATH = "data"

WORK_FOLDER_PATH = "workdir"

SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "Aircraft_reference_data.xml")

## Sizing of the aircraft

For the first part of this notebook, you will need to verify the sizing of your aircraft using FAST-OAD. To do so we will use a different way to compute the mission that the one used in the tutorials. In the tutorial, a simple Breguet formula was used, here we will use the mission file (It was possible to use in the first tutorial, a comment made mention of it).

### 1.1. The mission file

You will use the mission module to compute the fuel consumption over a specified mission. Therefore, you need to define the different phases your mission is composed of. That operation is done through the YAML [mission file](./data/design_mission.yml). If you have a closer look to this file, you will notice that it starts with a definition of the different flight phases (initial climb, climb, descent, holding, taxi) for which one or several flight parameter(s) is (are) varying.

For exemple, for the initial climb phase, the first segment is performed at iso-speed with a variation of altitude up to 400ft, then a second segment defines an acceleration at iso-altitude up to 250kt and a third segment at iso-speed and targetting an altitude of 15000ft completes this phase.

Each phases are then organised and completed with a cruise phase to form a route. Several routes can then be combined to form a mission profile. 

Each mission profile should have a unique name.

For further information about the mission definition, please see the [official documentation](https://fast-oad.readthedocs.io/en/documentation/mission_module/index.html).

### 1.2. The configuration file

As already mentioned, the [configuration file](./data/oad_sizing.yml) you are using enables you run the full OAD sizing process. Here a particular attention is paid on the performance module definition. 

First, in this section, you will call the [mission file](./data/design_mission.yml) you want to use through the `mission_file_path` option. You will then indicate which mission profile to select accordingly to the naming convention you chose in the mission file. This is done using the `mission_name` option. In the present situation you will compute performances for the mission called `MTOW_mission`. 

Note also that the option `is_sizing` of the performance group is active. That means that the mission TOW computed by the performance module is considered as the design Maximum Take-Off Weight and will impact the aircraft design. 

You can also notice that the option `add_solver` is deactivated as solvers are already defined for the whole problem in the [configuration file](./data/oad_sizing.yml).

In [4]:
OAD_CONFIGURATION_FILE = pth.join(DATA_FOLDER_PATH, "oad_sizing.yml")

### 1.3. The input file
You can now generate the input file and fill out the values that corresponds to your aircraft. If you look into the file you will see that some of the value asked are not known, you can keep the default value for most of them. However, you need to fill out the values in the following cell so that the process does not take the value rather than the one that correspond to your aircraft 

In [ ]:
input_file = oad.generate_inputs(OAD_CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

# TLARs
input_file["data:TLAR:NPAX"] = 
input_file["data:TLAR:approach_speed"] = 
input_file["data:TLAR:cruise_mach"] = 
input_file["data:TLAR:range"] = 

# Geometry
input_file["data:geometry:cabin:aisle_width"] = 
input_file["data:geometry:cabin:seats:economical:count_by_row"] = 
input_file["data:geometry:cabin:seats:economical:length"] = 
input_file["data:geometry:cabin:seats:economical:width"] = 

input_file["data:geometry:flap:chord_ratio"] = 
input_file["data:geometry:flap:span_ratio"] = 

input_file["data:geometry:horizontal_tail:aspect_ratio"] = 
input_file["data:geometry:horizontal_tail:sweep_25"] = 
input_file["data:geometry:horizontal_tail:taper_ratio"] = 
input_file["data:geometry:horizontal_tail:thickness_ratio"] = 

input_file["data:geometry:propulsion:engine:y_ratio"] = 

input_file["data:geometry:slat:chord_ratio"] = 
input_file["data:geometry:slat:span_ratio"] = 

input_file["data:geometry:vertical_tail:aspect_ratio"] = 
input_file["data:geometry:vertical_tail:sweep_25"] = 
input_file["data:geometry:vertical_tail:taper_ratio"] = 
input_file["data:geometry:vertical_tail:thickness_ratio"] = 

input_file["data:geometry:wing:aspect_ratio"] = 
input_file["data:geometry:wing:sweep_25"] = 
input_file["data:geometry:wing:kink:span_ratio"] = 
input_file["data:geometry:wing:MAC:at25percent:x"] = 

# Propuslion
input_file["data:propulsion:MTO_thrust"] = 

input_file["data:propulsion:rubber_engine:design_altitude"] =
input_file["data:propulsion:rubber_engine:maximum_mach"] =
input_file["data:propulsion:rubber_engine:overall_pressure_ratio"] =
input_file["data:propulsion:rubber_engine:turbine_inlet_temperature"] =

# Weight
input_file["data:weight:aircraft:max_payload"] = 
input_file["data:weight:aircraft:payload"] = 

# Mission
inputs_file["data:mission:MTOW_mission:diversion:distance"] = 
inputs_file["data:mission:MTOW_mission:holding:duration"] = 
inputs_file["data:mission:MTOW_mission:main_route:range"] = 
inputs_file["data:mission:MTOW_mission:takeoff:V2"] = 
inputs_file["data:mission:MTOW_mission:takeoff:fuel"] = 

inputs_file["data:mission:sizing:takeoff:flap_angle"] = 
inputs_file["data:mission:sizing:takeoff:slat_angle"] = 
inputs_file["data:mission:sizing:landing:flap_angle"] = 
inputs_file["data:mission:sizing:landing:slat_angle"] = 
inputs_file["data:mission:sizing:main_route:cruise:altitude"] = 

### 1.4. Run the OAD process

Now, run the problem and check the output file: